    Mitchell Probst
    Math 439
    Inverted Pendulum

In [ ]:
import numpy as np

import gym
import acme_gym
import time

from matplotlib import pyplot as plt
from scipy.optimize import root
from scipy import linalg
from scipy.integrate import odeint

# Problem 1

In [ ]:
g = 9.8

def linearized_init(M, m, l, q1, q2, q3, q4, r):
    """
    Parameters
    ----------
    M, m: floats
        masses of the rickshaw and the present
    l   : float
        length of the rod
    q1, q2, q3, q4, r : floats
        relative weights of the position and velocity of the rickshaw, the
        angular displacement theta and the change in theta, and the control
        
    Return
    ------
    A : ndarray of shape (4,4)
    B : ndarray of shape (4,1)
    Q : ndarray of shape (4,4)
    R : ndarray of shape (1,1)
    """
#     A = np.zeros((4,4))
#     A[0,1] = 1
#     A[1,2] = m*g/M
#     A[2,3] = 1
#     A[3,2] = g/(M*l)*(M + m)
    
#     B = np.zeros((4,1))
#     B[1] = 1/M
#     B[3] = 1/(M*l)

    A = np.zeros((4,4))
    A[1,2] = -12*m*g/(13*M + m)
    A[0,1] = 1
    A[2,3] = 1
    A[3,2] = 12*(m*g + M*g)/(l*(13*M + m))
    
    B = np.zeros((4,1))
    B[1] = 13/(13*M + m)
    B[3] = -12/(l*(13*M + m))
    
    #B[1] = (1 - m/(1/3 * (4*M + m)))/(M + m)
    #B[3] = -1 / (l/3 * (4*M + m))
    
    Q = np.diag([q1, q2, q3, q4])
    
    R = np.ones(1)*r
    
    return A,B,Q,R

# Problem 2

In [ ]:
def find_P(A, B, Q, R):
    """
    Find P where P is in terms of the Riccati equation
    
    Parameters
    ----------
    A : ndarray of shape (4,4)
    B : ndarray of shape (4,1)
    Q : ndarray of shape (4,4)
    R : ndarray of shape (1,1)
    
    Returns
    -------
    P : Matrix soloution of the Riccati equation
    """
    def root_func(p):
        """
        Gets passed a vector of ps
        """
        P = np.reshape(p, (4,4))
        return np.reshape(P @ A + A.T @ P + Q - 1/R * (P @ B @ B.T @ P), -1)
    
    # Initial Guess for P
    P = np.ones(16)
    P = root(root_func, P)
    return np.reshape(P.x, (4,4))

In [ ]:
# Mass of cart and masspole
M, m = 1, .1
l = 4.
# These represent weights on x, x', theta, theta prime.
# We WANT theta to be 0, so give that higher weight
q1, q2, q3, q4 = 1., 1., 10., 1.
r = 10.
A,B,Q,R = linearized_init(M, m, l, q1, q2, q3, q4, r)
P = find_P(A,B,Q,R)
P

In [ ]:
result = A - 1/R[0] * B @ B.T @ P
eigs = np.linalg.eig(result)
eigs[0]

We see that we have one real eigenvalue, which indicates this problem might be unstable!

In [ ]:
newP = linalg.solve_continuous_are(A,B,Q,R)
result = A - 1/R[0] * B @ B.T @ newP
eigs = np.linalg.eig(result)
eigs[0]

Using this fancy method, we see that all of our eigenvalues are negative

# Problem 3

In [ ]:
def rickshaw(tv, X0, A, B, Q, R, P):
    '''
    Parameters:
    ----------
    tv  : ndarray of time values, with shape (n+1,)
    X0  : Initial conditions on state variables
    A, Q: ndarrays of shape (4,4)
    B   : ndarray of shape (4,1)
    R   : ndarray of shape (1,1)
    P   : ndarray of shape (4,4)
    Returns
    -------
    Z : ndarray of shape (n+1,4), the state vector at each time
    U : ndarray of shape (n+1,), the control values
    '''
    n = np.shape(tv)[0]
    Z = np.zeros((n, 4))
    U = np.zeros((n))
    
    def find_z(z, t):
        return (A - 1/R * B @ B.T @ P) @ z

    Z = odeint(find_z, X0, tv)
    for t in range(n):
        U[t] = -1/R * B.T @ P @ Z[t]
        
    return Z, U

# Problem 4

In [ ]:
env = gym.make('CartPoleContinuous-v0')
#env.seed(1)
#env.theta_threshold_radians = 45 * np.pi / 180
#env.x_threshold = 5
obs = env.reset()


# Set up using initial values given, and choose N
M, m = 1., .1
l = 1.
q1, q2, q3, q4, = 1, 1, 10, 1
r = 10.

N = 100
tf = .02*N

A,B,Q,R = linearized_init(M, m, l, q1, q2, q3, q4, r)
P = linalg.solve_continuous_are(A,B,Q,R)
ts = np.linspace(0,tf,N+1)
Z, U = rickshaw(ts, obs, A, B, Q, R, P)

In [ ]:
obs, ts[:5], U[:5]

In [ ]:
array([-0.02895248,  0.02119629,  0.00193978, -0.0246696 ]

In [ ]:
U[:10]

In [ ]:
obs = env.reset()
N = 20
for i in range(N):
    #solve the problem
    Z, U = rickshaw(ts, obs, A, B, Q, R, P)
    for u in U[:10]:
        obs, rew, done, info = env.step(np.array([u]))
        env.render()
env.close()
env = gym.make('CartPoleContinuous-v0')
# while True:
#     step = spU[i]
#     time.sleep(.002)
#     i += 1
#     j += 1
#     if end is True:
#         step = 0.1
#     obs, rew, done, inf = env.step(np.array([step]))
#     gym_obs[j] = obs
#     env.render()
#     if done is True:
#         print("It went this many frames", j)
#         print("We are trying to end:", end)
#         q = input("Pause")
#         if q == 'q':
#             break
#         print("We are done, we fell at", i)
        
#     if i % 10 == 0:
#         sp_P = linalg.solve_continuous_are(A,B,Q,R)
#         sp_ts = np.linspace(0,tf2,N)
#         spZ, spU = rickshaw(sp_ts, obs, A, B, Q, R, sp_P)
#         i = 0
#         #q = input("Quit with 'q', all else continues")
#         #if q == 'q':
#         #    break
#     if j >= 50000:
#         #q = input("Quit with 'q', all else continues")
#         end = True
#         #if q == 'q':
#         #    break
#     if done is True:
#         q = input("Quit with 'q', all else continues")
#         if q == 'q':
#             break
# env.close()
# env = gym.make('CartPoleContinuous-v0')

In [ ]:
env2 = gym.make('CartPoleContinuous-v0')
#env.seed(1)
#env.theta_threshold_radians = 45 * np.pi / 180
#env.x_threshold = 5
init2 = env2.reset()


# Set up using initial values given, and choose N
M, m = 1., .1
l = 1.
q1, q2, q3, q4, = 1, 1, 10, 1
r = 10.

M = 100
tf2 = .02*M

A2,B2,Q2,R2 = linearized_init(M, m, l, q1, q2, q3, q4, r)
P2 = linalg.solve_continuous_are(A2,B2,Q2,R2)
ts2 = np.linspace(0,tf2,M+1)
Z2, U2 = rickshaw(ts2, init2, A2, B2, Q2, R2, P2)

In [ ]:
init2, ts2[:5], U2[:5]

In [ ]:
obs

In [ ]:
U[:10]

In [ ]:
obs = env.reset()
N = 20
for i in range(N):
    #solve the problem
    print("Solving for the {}th time".format(i))
    Z, U = rickshaw(ts, obs, A, B, Q, R, P)
    for u in U[:10]:
        obs, rew, done, info = env.step(np.array([u]))
        env.render()
env.close()
env = gym.make('CartPoleContinuous-v0')

In [ ]:
# Plot our solutions to see instability
fig, ax = plt.subplots(ncols=2, figsize=(12,8))
titles = ["scipy.optimize.root", "solve_continuous_are"]
for a,z,u,t,title in zip(ax, [Z,spZ], [U,spU], [ts,sp_ts], titles):
    a.plot(t,z[:,0], label="x")
    a.plot(t,z[:,1], label=r"$\dot{x}$")
    a.plot(t,z[:,2], label=r"$\theta$")
    a.plot(t,z[:,3], label=r"$\dot{\theta}$")
    a.plot(t,u, label="u")
    a.legend(loc='best')
    a.set_title(title)
ax[1].set_ylim(-6,6)
ax[1].set_xlim(0,tf2)
ax[0].set_xlim(0,tf1)
ax[0].set_ylim(-60,20)
plt.show()

If you run the first one for longer time you see the solutions continue to explode out towards +/- infinity